In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import numpy as np
import pandas as pd
import pathlib


In [ ]:
# Fazer o download do dataset Flowers utilizando o Keras
# As imagens ficam guardadas localmente em disco

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='flower_photos', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
# Verificar a diretoria em que se encontra o dataset
data_dir

In [ ]:
#Número total de imagens

total = len(list(data_dir.glob('*/*.jpg')))
print(total)

In [ ]:
# Criação do Dataset a partir das imagens em disco
# Este dataset ainda não está dividido. Em outros casos, já existem diretorias para os vários conjuntos

# Neste exemplo são usados métodos do keras.preprocessing para carregar as imagens para o tf.data.dataset
# Ver outras alternativas aqui: https://www.tensorflow.org/tutorials/load_data/images

# Define Batch size e dimensões das imagens
# Não esquecer que as imagens têm 3 canais (RGB)
# Cria conjunto de treino (80%) e de validação (20%)

batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
# Obter alguma informação sobre o dataset

#Cardinalidade 
print('Cardinalidade Treino: ', train_ds.cardinality().numpy())
print('Cardinalidade Validacão: ', val_ds.cardinality().numpy())

class_names = train_ds.class_names

print('Classes: ', class_names)


# Explicar o valor obtido para a cardinalidade dos conjuntos 


In [ ]:
#Mostrar algumas imagens do Dataset

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
     for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
# Obter 2 imagens para análise mais detalhada 

for images, labels in train_ds.take(1):
    im1 = images[0].numpy().astype("uint8")
    label1 = labels[0]
    im2 = images[1].numpy().astype("uint8")
    label2 = labels[1]
    
    
plt.figure(figsize=(10, 10))

ax = plt.subplot(1, 2, 1)
plt.imshow(im1)
plt.title(class_names[label1])
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(im2)
plt.title(class_names[label2])
plt.axis("off")

In [ ]:
# Converte para Float e normaliza para intervalo [0, 1)
im1_B = tf.image.convert_image_dtype(im1, dtype=tf.float32)

# Ajusta para tensor com 4 dimensões (Batch + RGB)
im1_B = tf.expand_dims(im1_B, axis=0)



In [ ]:
# Formato do tensor que vai entrar como input na CNN
# A primeira dimensão é o batch size

im1_B.shape

In [ ]:
# Criar filtros verticais e horizontais 

filter1 = np.zeros(shape=(7,7,3, 1), dtype = np.float32)

filter2 = np.zeros(shape=(7,7,3,1), dtype = np.float32)

filter1[: , 3 , : ,] = 1
filter2[3 , : , : ,] = 1


In [ ]:


a = filter2.squeeze()
R = a[:, :, 0]
G = a[:, :, 1]
B = a[:, :, 2]



plt.figure(figsize=(10, 10))

ax = plt.subplot(1, 3, 1)
plt.imshow(R)
plt.title("R")
plt.axis("off")
ax = plt.subplot(1, 3, 2)
plt.imshow(G)
plt.title("G")
plt.axis("off")
ax = plt.subplot(1, 3, 3)
plt.imshow(B)
plt.title("B")
plt.axis("off")



In [ ]:

# Escrever código para visualizar o filtro vertical





In [ ]:
# Aplicar CNN a cada uma das imagens
# Neste caso está a ser usado o método diretamente da biblioteca TensorFlow - Não estamos a usar a API Keras
# Ainda não está a ser aplicada a função de ativação

output1 = tf.nn.conv2d(im1_B, filter1, strides=1, padding="SAME")
output2 = tf.nn.conv2d(im1_B, filter2, strides=1, padding="SAME")

In [ ]:
output1.shape

In [ ]:
plt.figure(figsize=(50, 50))

ax = plt.subplot(1, 2, 1)
plt.imshow(output1[0,:,:], cmap="binary")
plt.title("Vertical", fontsize=30)
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(output2[0,:,:], cmap="binary")
plt.title("Horizontal", fontsize=30)
plt.axis("off")

# Analisar os padrões que se salientam nas imagens obtidas


In [ ]:
# Aplicar função de ativação RELU

image1 = tf.nn.relu(output1)
image2 = tf.nn.relu(output2)

#x = image1[0,:,:]
x = tf.squeeze(image1)
x.shape


In [ ]:
plt.figure(figsize=(50, 50))

ax = plt.subplot(1, 2, 1)
plt.imshow(tf.squeeze(image1), cmap="binary")
plt.title("Vertical", fontsize=30)
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(image2[0,:,:], cmap="binary")
plt.title("Horizontal", fontsize=30)
plt.axis("off")



In [ ]:
# Exercício: Qual a dimensão dos mapas de outputs em cada uma destas 3 situações?
# Dimensões de input: Imagem (180, 180, 3), Filtro (7, 7, 3)

outputE1 = tf.nn.conv2d(im1_B, filter1, strides=2, padding="SAME")

outputE2 = tf.nn.conv2d(im1_B, filter1, strides=1, padding="VALID")

outputE3 = tf.nn.conv2d(im1_B, filter1, strides=2, padding="VALID")



In [ ]:
# Confirmar se as previsões estão corretas, verificando o formato dos outputs


In [ ]:
# Aplicação de uma camada MaxPool

pool1 = tf.nn.max_pool(image1, 2, strides=2, padding="SAME")
pool2 = tf.nn.max_pool(image2, 2, strides=2, padding="SAME")

In [ ]:
#Confirmar o Downsampling da imagem

pool1.shape

In [ ]:
plt.figure(figsize=(20, 20))

ax = plt.subplot(1, 2, 1)
plt.imshow(tf.squeeze(pool1), cmap="binary")
plt.title("Vertical", fontsize=30)
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(tf.squeeze(pool2), cmap="binary")
plt.title("Horizontal", fontsize=30)
plt.axis("off")

In [ ]:
# Criação da CNN para classificação
# Vai ser criado um modelo sequencial em que as camadas se sucedem umas às outras
# https://www.tensorflow.org/guide/keras/sequential_model


model = keras.Sequential([
    layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=[180, 180, 3]),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu',padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu',padding='same'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')
])


In [ ]:
# Exercício: Descrever as características desta CNN

# Quais são as camadas de extração de features?

# Quais são as camadas de classificação?



In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
#Obter detalhes da segunda camada CNN (3ª camada da rede)

my_layer = model.layers[2]
my_layer.name


In [ ]:
weights, biases = my_layer.get_weights()
print("Weights Shape: ", weights.shape)
print("Biases Shape: ", biases.shape)


In [ ]:
# Explicar os valores obtidos


In [ ]:
# Compilar o modelo
# https://www.tensorflow.org/api_docs/python/tf/keras/Model
# 1. Definir a função de loss: https://keras.io/api/losses/
# 2. Definir o algoritmo de otimização/aprendizagem: https://keras.io/api/optimizers/
# Definir a métrica de avaliação

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

In [ ]:
# Visualização da evolução da accurary e da loss

import pandas as pd 
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
# O que pode ser concluído das curvas em cima?

# O que fazer para melhorar o desempenho?

# Alterar a CNN, por forma a tentar melhorar o seu desempenho

In [ ]:
# Experimentar uma pequena fase de teste
# Vamos usar algumas imagens e ver qual a classificação 

# Primeiro vamos usar uma imagem exemplo

# Fazer o download da imagem e efetuar o pré-processamento

sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = keras.preprocessing.image.load_img(sunflower_path, target_size=(img_height, img_width))

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) 


In [ ]:
# Aplicar o modelo e obter a previsão

predictions = model.predict(img_array)


In [ ]:
print(predictions)

print(predictions.sum())

In [ ]:
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(predictions)], 100 * np.max(predictions))
)

In [ ]:
plt.imshow(img)

In [ ]:
# Repetir o processo para uma imagem à vossa escolha
# Deve ser colocada na diretoria de trabalho onde têm o notebook

nomeIm = 'ex.jpg'

imX = tf.keras.preprocessing.image.load_img(nomeIm, target_size=(img_height, img_width))

In [ ]:
plt.imshow(imX)

In [ ]:
# Repetir o processo para a nova imagem

imX_array = keras.preprocessing.image.img_to_array(imX)
imX_array = tf.expand_dims(imX_array, 0) # Create a batch

In [ ]:
predictions = model.predict(imX_array)

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(predictions)], 100 * np.max(predictions))
)


In [ ]:
predictions